# 00 — Universe Provider (tickers from package or file)

This notebook explains the new **Universe Provider** concept.

Instead of passing tickers manually every day, you can:
- load a predefined universe shipped with the package (CSV inside the repo/package)
- load your own universe from a local file
- optionally cap the amount of tickers (`--top` / `max_tickers`) to speed up downloads

Key idea:
> Universe Provider returns the list of tickers to analyze, and ensures the benchmark (SPY) is included.

In [1]:
import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 80)

## 1) Load a universe from the package (e.g. `mega`)

In [2]:
from swing_screener.data.universe import load_universe_from_package, UniverseConfig

tickers = load_universe_from_package("mega", UniverseConfig(max_tickers=50))
tickers[:15], len(tickers)

(['AAPL',
  'MSFT',
  'NVDA',
  'AMZN',
  'META',
  'GOOGL',
  'TSLA',
  'AMD',
  'AVGO',
  'NFLX',
  'INTC',
  'SPY'],
 12)

Notice SPY is included (required for Relative Strength).

In [3]:
"SPY" in tickers

True

## 2) Cap the universe size (like CLI `--top`)

In [4]:
tickers10 = load_universe_from_package("mega", UniverseConfig(max_tickers=10))
tickers10, len(tickers10)

(['AAPL',
  'MSFT',
  'NVDA',
  'AMZN',
  'META',
  'GOOGL',
  'TSLA',
  'AMD',
  'AVGO',
  'SPY'],
 10)

## 3) Load a universe from a local file (`--universe-file`)

Create a small universe file in the notebook (for demo).  
Real usage: you keep a `my_tickers.csv` somewhere and point the CLI to it.

In [5]:
from pathlib import Path

p = Path("my_tickers.csv")
p.write_text("AAPL\nMSFT\nNVDA\nAMZN\n# comment\nMETA\n", encoding="utf-8")
str(p)

'my_tickers.csv'

In [6]:
from swing_screener.data.universe import load_universe_from_file

tickers_file = load_universe_from_file("my_tickers.csv", UniverseConfig(max_tickers=None))
tickers_file

['AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'SPY']

## 4) Fetch data using universe tickers

In [7]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

ohlcv = fetch_ohlcv(tickers10 + ["SPY"], MarketDataConfig(start="2023-01-01"))
ohlcv.head()

Open                                                                                                               High  \
                  AAPL        MSFT       NVDA       AMZN        META      GOOGL        TSLA        AMD       AVGO         SPY        AAPL   
Date                                                                                                                                        
2023-01-03  128.343772  237.403930  14.836148  85.459999  121.968015  88.918144  118.470001  66.000000  54.020365  369.481572  128.954553   
2023-01-04  125.004155  226.856104  14.552432  86.550003  126.496383  89.672461  109.110001  65.000000  53.828179  368.337652  126.747853   
2023-01-05  125.240599  221.894726  14.476508  85.330002  125.255039  86.814062  110.510002  64.150002  53.132138  366.934233  125.871086   
2023-01-06  124.137217  217.792823  14.459526  83.029999  128.075353  86.139150  103.000000  63.150002  53.840617  367.789699  128.353598   
2023-01-09  128.530934  221.162221  15.268716  87.459999  130.250163  87.697379  118.959999  66.220001  56.630547  375.249150  131.427242   

                                                                                                                          Low              \
                  MSFT       NVDA       AMZN        META      GOOGL        TSLA        AMD       AVGO         SPY        AAPL        MSFT   
Date                                                                                                                                        
2023-01-03  240.011582  14.981004  86.959999  125.493393  90.367202  118.800003  66.879997  54.266086  371.461777  122.324579  231.856553   
2023-01-04  227.432323  14.838145  86.980003  128.154804  89.970214  114.589996  65.790001  54.322491  370.933080  123.221057  220.683688   
2023-01-05  222.236560  14.549435  85.419998  127.628467  86.913310  111.750000  64.349998  53.527014  367.049580  122.905826  216.581752   
2023-01-06  220.488370  14.994990  86.400002  129.425920  87.032402  114.389999  64.300003  56.506257  374.172514  123.033860  214.228059   
2023-01-09  225.840379  16.039943  89.480003  132.027740  89.374707  123.519997  69.320000  57.526425  378.450180  127.959553  221.123161   

                                                                                                            Close                         \
                 NVDA       AMZN        META      GOOGL        TSLA        AMD       AVGO         SPY        AAPL        MSFT       NVDA   
Date                                                                                                                                       
2023-01-03  14.081903  84.209999  121.431760  87.856169  104.639999  63.590000  52.661727  363.194888  123.211205  233.985657  14.300684   
2023-01-04  14.226758  83.360001  124.976998  86.615556  107.519997  63.310001  53.162725  365.280836  124.482033  223.750366  14.734249   
2023-01-05  14.133851  83.070000  123.676073  85.255835  107.160004  62.299999  52.693283  364.088896  123.161957  217.118912  14.250734   
2023-01-06  14.019966  81.430000  125.165678  84.223623  101.809998  60.049999  53.762215  364.713667  127.693558  219.677750  14.844141   
2023-01-09  15.125858  87.080002  128.383200  87.201128  117.110001  65.669998  53.595846  372.653751  128.215683  221.816574  15.612371   

                                                                                               Volume                                 \
                 AMZN        META      GOOGL        TSLA        AMD       AVGO         SPY       AAPL      MSFT       NVDA      AMZN   
Date                                                                                                                                   
2023-01-03  85.820000  123.874695  88.451675  108.099998  64.019997  52.918922  366.069092  112117500  25740000  401277000  76706000   
2023-01-04  85.139999  126.486458  87.419487  113.639999  64.660004  53.565247  368.895203   89

If you get rate limits or slow downloads, reduce universe size via `max_tickers`
or use caching in `fetch_ohlcv`.